## Sequence Tagging using Sequential Models

Sequence Tagging is an information extraction technique to identify and classify named entities in text. These entities can be pre-defined and generic like location names, organizations, time and etc...

In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

#### Desired Sample Output

<img src="https://miro.medium.com/max/2400/1*8LOMipM-fmszClg-AwATkQ.png">

### Files required are given in below link

https://drive.google.com/drive/folders/1m9JjfsAEN50flYwFPCgZWQ5nHXGt0ZwI?usp=sharing



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")

from google.colab import drive
drive.mount('/content/gdrive/')

project_path = "/content/gdrive/My Drive/NLP/"
data_filename = "ner_dataset.csv.zip"

import zipfile
z = zipfile.ZipFile(project_path + data_filename, 'r')
z.extractall()
z.close()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
pwd

'/content'

In [0]:
data = pd.read_csv("ner_dataset.csv", encoding="latin-1")
# data = data.drop(['POS'], axis =1)
# data = data.fillna(method="ffill")
data.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


### Fill the NaN with Sentence tag to easily identify the words of a single sentence.

In [0]:
data = data.fillna(method="ffill")

### Drop POS column from dataset as we are only interested in tags for sentence tagging.

In [0]:
data = data.drop(['POS'], axis =1)

In [0]:
data.head(30)

,Sentence #,Word,Tag
0,Sentence: 1,Thousands,O
1,Sentence: 1,of,O
2,Sentence: 1,demonstrators,O
3,Sentence: 1,have,O
4,Sentence: 1,marched,O
5,Sentence: 1,through,O
6,Sentence: 1,London,B-geo
7,Sentence: 1,to,O
8,Sentence: 1,protest,O
9,Sentence: 1,the,O


Now we can see from the above result that the same sentence words have same sentence id.

In [0]:
tags = list(set(data["Tag"].values))
n_tags = len(tags)
n_tags

17

In [0]:
print(tags)

['B-org', 'B-geo', 'B-nat', 'I-geo', 'B-tim', 'I-gpe', 'B-art', 'B-eve', 'B-per', 'O', 'I-nat', 'I-per', 'I-eve', 'I-art', 'I-tim', 'I-org', 'B-gpe']


In [0]:
words = set(list(data['Word'].values))
words.add('dummy')
n_words = len(words)
n_words

35179

### Groupby sentences and combining words and tags for each setence using groupby and apply on dataframe

In [0]:
combining_words_tags = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
d = data.groupby("Sentence #").apply(combining_words_tags)

In [0]:
sentences = [s for s in d]

In [0]:
sentences[0]

[('Thousands', 'O'),
 ('of', 'O'),
 ('demonstrators', 'O'),
 ('have', 'O'),
 ('marched', 'O'),
 ('through', 'O'),
 ('London', 'B-geo'),
 ('to', 'O'),
 ('protest', 'O'),
 ('the', 'O'),
 ('war', 'O'),
 ('in', 'O'),
 ('Iraq', 'B-geo'),
 ('and', 'O'),
 ('demand', 'O'),
 ('the', 'O'),
 ('withdrawal', 'O'),
 ('of', 'O'),
 ('British', 'B-gpe'),
 ('troops', 'O'),
 ('from', 'O'),
 ('that', 'O'),
 ('country', 'O'),
 ('.', 'O')]

In [0]:
print(len(sentences))
sentences = sentences[:3200]

47959


### Map words and tags to integers

In [0]:
words2index = {w:i for i,w in enumerate(words)}
tags2index = {t:i for i,t in enumerate(tags)}
print(words2index['India'])
print(tags2index['B-geo'])

27661
1


#### Make all sentences equal length by appending a `dummy` token at the end of the sentence if the sentence is short. And if the sentence is long consider only `max_length` number of words from that sentence. 

In [0]:
max_len = 30
X = [[w[0]for w in s] for s in sentences]
new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("dummy")
    new_X.append(new_seq)
new_X[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.',
 'dummy',
 'dummy',
 'dummy',
 'dummy',
 'dummy',
 'dummy']

#### Similarly pad labels with `O` tag

In [0]:
from keras.preprocessing.sequence import pad_sequences
y = [[tags2index[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tags2index["O"])
y[0]

array([ 9,  9,  9,  9,  9,  9,  1,  9,  9,  9,  9,  9,  1,  9,  9,  9,  9,
        9, 16,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9], dtype=int32)

### Split the dataset into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(new_X, y, test_size=0.2, random_state=10)

In [0]:
len(X_tr)

2560

In [0]:
np.array(X_tr).shape

(2560, 30)

In [0]:
np.array(y_tr).shape

(2560, 30)

### Using ELMO(Deep contextualized word representations) embeddings (like word2vec) as pre-trained word embeddings

Link to paper: https://arxiv.org/pdf/1802.05365.pdf

Importing these embeddings from tensorflow hub as given in this link - https://tfhub.dev/google/elmo/1

In [0]:
batch_size = 32
# import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
sess = tf.Session()
K.set_session(sess)

In [0]:
tf.__version__, hub.__version__

('1.15.0', '0.7.0')

In [0]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)

In [0]:
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

## Model

In [0]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda, Flatten

In [0]:
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=50, return_sequences=True, dropout=0.2))(embedding)
# x = Bidirectional(LSTM(units=50, return_sequences=True))(x)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
def custom_sparse_categorical_accuracy(y_true, y_pred):
    return K.cast(K.equal(K.max(y_true, axis=-1),
                          K.cast(K.argmax(y_pred, axis=-1), K.floatx())),
                  K.floatx())

In [0]:
model = Model(input_text, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[custom_sparse_categorical_accuracy])

In [0]:
# y_tr = (np.arange(y_tr.max()+1) == y_tr[...,None]).astype(int)
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)

In [0]:
y_te = y_te.reshape(y_te.shape[0], y_te.shape[1], 1)

In [0]:
y_tr.shape

(2560, 30, 1)

In [0]:
y_te.shape

(640, 30, 1)

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30)                0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 30, 1024)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 30, 100)           430000    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 17)            1717      
Total params: 431,717
Trainable params: 431,717
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = model.fit(np.array(X_tr), y_tr, batch_size=batch_size, epochs=1, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1



2560/2560 [==============================] - 29s 11ms/step - loss: 0.4661 - custom_sparse_categorical_accuracy: 0.8931


In [0]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=d0f08427840e7376d9b178c41672255a28dda1b86515f609c5823678b6c6c7e3
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
np.array(X_te[:]).shape

(640, 30)

In [0]:
idx2tag = {i: w for w, i in tags2index.items()}

In [0]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

test_pred = model.predict(np.array(X_te[:32*10]), verbose=1)

idx2tag = {i: w for w, i in tags2index.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("dummy", "O"))
        out.append(out_i)
    return out

320/320 [==============================] - 2s 6ms/step


In [0]:
def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p[0]].replace("dummy", "O"))
        out.append(out_i)
    return out

In [0]:
pred_labels = pred2label(test_pred)

In [0]:
test_labels = test2label(y_te[:32*10])
print(classification_report(test_labels, pred_labels))

           precision    recall  f1-score   support

      per       0.40      0.39      0.40        95
      art       0.00      0.00      0.00         4
      geo       0.55      0.59      0.57       242
      gpe       0.66      0.62      0.64        97
      org       0.16      0.06      0.08       141
      tim       0.59      0.52      0.55       138
      nat       0.00      0.00      0.00         2
      eve       0.00      0.00      0.00         2

micro avg       0.52      0.44      0.48       721
macro avg       0.47      0.44      0.45       721



The score can be increased by considering full train data and increasing the epochs.

### To visualize tags on text

In [0]:
!pip install ipymarkup

In [0]:
from ipymarkup import show_box_markup
from ipymarkup.palette import palette, BLUE, RED, GREEN, ORANGE, PURPLE

In [0]:
test = X_te[2]
text = ' '.join(test)

spans = []
current_pos2 = 0
for index, i in enumerate(test):
  if index<max_len and i !="dummy":
    if len(idx2tag[y_te[2][index][0]].split('-'))>1:
      tag = idx2tag[y_te[2][index][0]].split('-')[1]
    else:
      tag = idx2tag[y_te[2][index][0]].split('-')[0]
    
    current_pos1 = current_pos2
    current_pos2 += len(i)+1
    if current_pos2 > current_pos1:
      spans.append( (current_pos1, current_pos2, tag) )

In [0]:
spans

[(0, 10, 'O'),
 (10, 14, 'O'),
 (14, 18, 'O'),
 (18, 23, 'O'),
 (23, 27, 'O'),
 (27, 36, 'O'),
 (36, 39, 'O'),
 (39, 43, 'O'),
 (43, 50, 'O'),
 (50, 54, 'O'),
 (54, 63, 'O'),
 (63, 73, 'tim'),
 (73, 76, 'O'),
 (76, 80, 'O'),
 (80, 88, 'O'),
 (88, 97, 'O'),
 (97, 105, 'O'),
 (105, 109, 'O'),
 (109, 116, 'geo'),
 (116, 123, 'O'),
 (123, 128, 'O'),
 (128, 135, 'O'),
 (135, 139, 'O'),
 (139, 146, 'gpe'),
 (146, 153, 'O'),
 (153, 155, 'O')]

In [0]:
show_box_markup(text, spans, palette=palette(tim=BLUE, geo=RED, gpe=ORANGE, O=PURPLE))

BoxMarkup('Officials say the bomb was attached to the tanker and exploded Wednesday as the vehicle traveled through the Khyber tribal area toward the Afghan border . dummy dummy dummy dummy',
          [Span(0, 10, 'O'),
           Span(10, 14, 'O'),
           Span(14, 18, 'O'),
           Span(18, 23, 'O'),
           Span(23, 27, 'O'),
           Span(27, 36, 'O'),
           Span(36, 39, 'O'),
           Span(39, 43, 'O'),
           Span(43, 50, 'O'),
           Span(50, 54, 'O'),
           Span(54, 63, 'O'),
           Span(63, 73, 'tim'),
           Span(73, 76, 'O'),
           Span(76, 80, 'O'),
           Span(80, 88, 'O'),
           Span(88, 97, 'O'),
           Span(97, 105, 'O'),
           Span(105, 109, 'O'),
           Span(109, 116, 'geo'),
           Span(116, 123, 'O'),
           Span(123, 128, 'O'),
           Span(128, 135, 'O'),
           Span(135, 139, 'O'),
           Span(139, 146, 'gpe'),
           Span(146, 153, 'O'),
           Span(153, 155, 'O')])